In [ ]:
import utils
import CNNModel
import torch
import argparse
import requests
import librosa

In [ ]:
import os

In [ ]:
num_classes = 3
model_path = './CNN_model.pth'
Labels = {'HAP': 0, 'NEU': 1, 'SAD': 2}

# parser = argparse.ArgumentParser(description='arguments')
# parser.add_argument('model', type=str, help='Model type')
# parser.add_argument('audio', type=str, help='audio.wav path')

# args = parser.parse_args()

In [ ]:
def predict(audio, model):
    testFeature = torch.tensor(utils.extract_features(audio, 'CNN').reshape(1, 1, 9, 18))
    out = model(testFeature.float())
    max_value = torch.argmax(out)
    result = utils.label_query(Labels, max_value.item())[0]
    # print(" Label predict:", result, '\n', 'Label expect: ', 'NEU') # Fixed Currently
    return result

In [ ]:
def get_tempo(fileName):
    # Load a built-in audio sample (example: Beat loop)
    y, sr = librosa.load(fileName)
    # Estimate the tempo and beat events
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
    return tempo

In [ ]:
def call_server(emotion_tag, tempo):
    # Sending request to Java Script Backend Server
    # emotion = emotion_tag # Important: This needs to be Calm, Happy, Energetic or Sad
    sever_labels = { 'HAP': 'Happy', 'NEU': 'Calm', 'SAD': 'Sad' }

    url = f"http://localhost:3004?emotion={sever_labels[emotion_tag]}&tempo={tempo}"
    headers = {}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for non-2xx status codes
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(e)

In [ ]:
# def main():
#     if args.model == 'CNN':
model = CNNModel.CNN()
utils.load_model(model, model_path)

audio = ["1001_IEO_HAP_HI.wav", "2663_NEU_XX.wav","imhappy.wav"]

# In order to change audio file, change the index of audio list 

emotion = predict(audio[1], model)
tempo = get_tempo(audio[1])

print(f"Predicted emotion: {emotion}")

# Call server here with requests
call_server(emotion, tempo)